In [2]:
!pip install faster-whisper deep-translator edge-tts
!pip install nest-asyncio
!apt-get install ffmpeg libass-dev -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libfribidi-dev libgraphite2-dev libharfbuzz-dev libharfbuzz-gobject0
  libharfbuzz-icu0
Suggested packages:
  libgraphite2-utils
The following NEW packages will be installed:
  libass-dev libfribidi-dev libgraphite2-dev libharfbuzz-dev
  libharfbuzz-gobject0 libharfbuzz-icu0
0 upgraded, 6 newly installed, 0 to remove and 37 not upgraded.
Need to get 787 kB of archives.
After this operatio

In [3]:
import subprocess
import asyncio
import nest_asyncio
import os
nest_asyncio.apply()

from google.colab import files
from faster_whisper import WhisperModel
from deep_translator import GoogleTranslator
import edge_tts

# ─────────────────────────────────────────
# CONFIG — change these as needed
# ─────────────────────────────────────────
TARGET_LANGUAGE = 'en'        # 'en' English, 'hi' Hindi, 'ta' Tamil, 'te' Telugu
TTS_VOICE = "en-IN-PrabhatNeural"  # Male Indian English
ADD_CAPTIONS = True           # Set False to skip captions
CAPTION_COLOR = "white"       # white or yellow

# ─────────────────────────────────────────
# STEP 1 — Upload video
# ─────────────────────────────────────────
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded: {video_path} ✅")

# ─────────────────────────────────────────
# STEP 2 — Extract audio
# ─────────────────────────────────────────
audio_path = "audio.wav"
subprocess.run(
    f"ffmpeg -i '{video_path}' -vn -acodec pcm_s16le -ar 16000 -ac 1 {audio_path} -y",
    shell=True, capture_output=True
)
print("Audio extracted ✅")

# ─────────────────────────────────────────
# STEP 3 — Transcribe WITH timestamps
# ─────────────────────────────────────────
print("Loading Whisper... (1-2 mins first time)")
model = WhisperModel("medium")
segments, info = model.transcribe(audio_path)
segments = list(segments)

print(f"Detected language: {info.language}")
print("\nSegments with timestamps:")
for s in segments:
    print(f"  [{s.start:.1f}s → {s.end:.1f}s]: {s.text}")

# ─────────────────────────────────────────
# STEP 4 — Translate each segment
# ─────────────────────────────────────────
translated_segments = []
for s in segments:
    translated = GoogleTranslator(source='auto', target=TARGET_LANGUAGE).translate(s.text)
    translated_segments.append({
        "start": s.start,
        "end": s.end,
        "original": s.text,
        "translated": translated
    })
    print(f"[{s.start:.1f}s → {s.end:.1f}s]: {translated}")

print(f"\n✅ Translated {len(translated_segments)} segments")

# ─────────────────────────────────────────
# STEP 5 — Generate TTS per segment + fit to timestamp
# ─────────────────────────────────────────
os.makedirs("chunks", exist_ok=True)

async def generate_chunk(text, filename):
    communicate = edge_tts.Communicate(text, voice=TTS_VOICE)
    await communicate.save(filename)

chunk_files = []

for i, seg in enumerate(translated_segments):
    raw_chunk = f"chunks/raw_{i}.mp3"
    fitted_chunk = f"chunks/fitted_{i}.mp3"

    asyncio.run(generate_chunk(seg["translated"], raw_chunk))

    result = subprocess.run(
        f"ffprobe -v error -show_entries format=duration "
        f"-of default=noprint_wrappers=1:nokey=1 '{raw_chunk}'",
        shell=True, capture_output=True, text=True
    )
    tts_duration = float(result.stdout.strip())
    target_duration = seg["end"] - seg["start"]

    speed_ratio = tts_duration / target_duration
    speed_ratio = max(0.5, min(2.0, speed_ratio))

    subprocess.run(
        f"ffmpeg -i '{raw_chunk}' -filter:a 'atempo={speed_ratio:.4f}' '{fitted_chunk}' -y",
        shell=True, capture_output=True
    )

    chunk_files.append({
        "file": fitted_chunk,
        "start": seg["start"],
        "duration": target_duration
    })
    print(f"Chunk {i} — TTS {tts_duration:.1f}s → fitted to {target_duration:.1f}s at {speed_ratio:.2f}x ✅")

print("\nAll chunks generated ✅")

# ─────────────────────────────────────────
# STEP 6 — Stitch chunks into full audio track
# ─────────────────────────────────────────
result = subprocess.run(
    f"ffprobe -v error -show_entries format=duration "
    f"-of default=noprint_wrappers=1:nokey=1 '{video_path}'",
    shell=True, capture_output=True, text=True
)
total_duration = float(result.stdout.strip())

silent_base = "silent_base.mp3"
subprocess.run(
    f"ffmpeg -f lavfi -i anullsrc=r=44100:cl=stereo -t {total_duration} '{silent_base}' -y",
    shell=True, capture_output=True
)

inputs = f"-i '{silent_base}'"
filter_parts = []
for i, chunk in enumerate(chunk_files):
    delay_ms = int(chunk["start"] * 1000)
    inputs += f" -i '{chunk['file']}'"
    filter_parts.append(f"[{i+1}:a]adelay={delay_ms}|{delay_ms}[a{i}]")

mix_inputs = "".join([f"[a{i}]" for i in range(len(chunk_files))])
filter_complex = ";".join(filter_parts) + f";[0:a]{mix_inputs}amix=inputs={len(chunk_files)+1}[out]"

final_audio = "final_audio.mp3"
subprocess.run(
    f"ffmpeg {inputs} -filter_complex \"{filter_complex}\" "
    f"-map \"[out]\" -t {total_duration} '{final_audio}' -y",
    shell=True
)
print("Audio stitched ✅")

# ─────────────────────────────────────────
# STEP 7 — Merge audio into video
# ─────────────────────────────────────────
final_video = "final_dubbed.mp4"
subprocess.run(
    f"ffmpeg -i '{video_path}' -i '{final_audio}' "
    f"-map 0:v -map 1:a -c:v copy -shortest '{final_video}' -y",
    shell=True
)
print("Final dubbed video ready ✅")

# ─────────────────────────────────────────
# STEP 8 — Generate SRT caption file
# ─────────────────────────────────────────
def generate_srt(segments, output_file="subtitles.srt"):
    def fmt(seconds):
        h = int(seconds // 3600)
        m = int((seconds % 3600) // 60)
        s = int(seconds % 60)
        ms = int((seconds % 1) * 1000)
        return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

    with open(output_file, 'w', encoding='utf-8') as f:
        for i, seg in enumerate(segments):
            f.write(f"{i+1}\n")
            f.write(f"{fmt(seg['start'])} --> {fmt(seg['end'])}\n")
            f.write(f"{seg['translated']}\n\n")

    print(f"SRT file generated ✅")
    return output_file

srt_path = generate_srt(translated_segments)

# ─────────────────────────────────────────
# STEP 9 — Burn captions into video
# ─────────────────────────────────────────
if ADD_CAPTIONS:
    captioned_video = "final_dubbed_captioned.mp4"

    color_code = "&H00ffffff" if CAPTION_COLOR == "white" else "&H0000ffff"

    subprocess.run(
        f"ffmpeg -i '{final_video}' "
        f"-vf \"subtitles={srt_path}:force_style='FontSize=16,"
        f"FontName=Arial Bold,"
        f"PrimaryColour={color_code},"
        f"OutlineColour=&H00000000,"
        f"BackColour=&H80000000,"
        f"Outline=1,"
        f"Shadow=1,"
        f"Alignment=2,"
        f"MarginV=30'\" "
        f"-c:a copy '{captioned_video}' -y",
        shell=True
    )
    print("Captions burned into video ✅")

    output_video = captioned_video
else:
    output_video = final_video

# ─────────────────────────────────────────
# STEP 10 — Download
# ─────────────────────────────────────────
print(f"\n📥 Downloading final video...")
files.download(output_video)
print("Downloaded ✅")

print("\n" + "="*50)
print("✅ DUBBING PIPELINE COMPLETE")
print(f"   Language: {TARGET_LANGUAGE}")
print(f"   Segments dubbed: {len(translated_segments)}")
print(f"   Captions: {'Added ✅' if ADD_CAPTIONS else 'Skipped'}")
print("="*50)

Saving reel1.mp4 to reel1.mp4
Uploaded: reel1.mp4 ✅
Audio extracted ✅
Loading Whisper... (1-2 mins first time)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Detected language: hi

Segments with timestamps:
  [0.0s → 4.0s]:  आप लोग के लिए टास्क है। इस पानी की बौटल को खाली करना है।
  [4.0s → 8.0s]:  लेकिन टॉिस्ट यह है, कि आपको इसको कम से कम समय में खाली कर के दिखाना है।
  [8.0s → 11.0s]:  ट्राइ करना चाहेंगे? आजो, स्टार्ट।
  [14.0s → 16.0s]:  बचाहे, बचाहे।
  [17.0s → 20.0s]:  स्टार्ट। ग्यारा सेकिन, तीके, एलेवन सेकिन्स।
  [20.0s → 25.0s]:  आप में से कुई और ट्राइ करना चाहेंगा? आजो, तीके, स्ट्री, यह क्या है?
  [27.0s → 28.0s]:  स्टार्ट।
  [30.0s → 31.0s]:  स्टार्ट।
  [33.0s → 37.0s]:  6.97 सेकिन्स, नहीं बाग, वोर्ट बढ़िया।
  [37.0s → 41.0s]:  अब एक बाद मैं ट्राइ करूँगा, लेकिन इसके लिए मैं थोड़ी सी फिजिक्स का इस्तमाल करूँगा।
  [47.0s → 49.0s]:  आप सब लुग टाइमर लगा लिए जे।
  [57.0s → 60.0s]:  फिजिक्स के ऐसा क्या कौनसेप लग गया, कि मेरी बौटर लिए ती जल्दी का लिए?
[0.0s → 4.0s]: There is a task for you guys. This water bottle has to be emptied.
[4.0s → 8.0s]: But the twist is, you have to clear it in minimum time.
[8.0s → 11.0s]: Want to try? Come on, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded ✅

✅ DUBBING PIPELINE COMPLETE
   Language: en
   Segments dubbed: 12
   Captions: Added ✅
